In [ ]:
# # !pip install scikit-optimize
# !pip install --upgrade tensorflow

In [7]:
# Step 1: Authenticate and set up the Google Cloud Storage client

auth.authenticate_user()


# Initialize the client
client = storage.Client()

# Step 2: Define the bucket name (replace 'covid19_dataset_bucket' with your actual bucket name)
bucket_name = 'covid19_dataset_bucket'  # Ensure this is correct and matches your cloud storage
bucket = client.get_bucket(bucket_name)

# Optional: List files in the bucket to verify the existence of your dataset
blobs = bucket.list_blobs()
print("Files in bucket:")
for blob in blobs:
    print(blob.name)

Files in bucket:
COVID-19_Radiography_Dataset.zip


In [8]:
# Step 3: Specify the path of the file in the bucket with the extra space
blob = bucket.blob('COVID-19_Radiography_Dataset.zip')  # Note the space before .zip

# Step 4: Download the zip file to Colab environment
blob.download_to_filename('COVID-19_Radiography_Dataset.zip')
print("Download completed.")

Download completed.


In [9]:


# Step 5: Unzip the downloaded file into the /content/ directory

with zipfile.ZipFile('COVID-19_Radiography_Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')
print("Unzipping completed.")

# Optional: Check if files were extracted successfully

print(os.listdir('/content/'))

Unzipping completed.
['.config', 'COVID-19_Radiography_Dataset.zip', '__MACOSX', 'COVID-19_Radiography_Dataset ', 'sample_data']


In [15]:

# Path to the folder with extra space
original_dir = '/content/COVID-19_Radiography_Dataset '

# # Correct the folder name by removing the extra space
# corrected_dir = '/content/COVID-19_Radiography_Dataset'
# os.rename(original_dir, corrected_dir)

# Verify the rename and list the contents again
print(os.listdir('/content/'))

# # List contents of the corrected dataset directory
# print(os.listdir(corrected_dir))

['.config', 'COVID-19_Radiography_Dataset.zip', '__MACOSX', 'COVID-19_Radiography_Dataset ', 'sample_data']


In [16]:
# Update the data_dir to reflect the correct path within Colab
data_dir = '/content/COVID-19_Radiography_Dataset '

# List the contents to confirm it's correct
print(os.listdir(data_dir))

['Normal', 'Viral Pneumonia.metadata.xlsx', 'Lung_Opacity', 'COVID', 'Normal.metadata.xlsx', 'Lung_Opacity.metadata.xlsx', 'COVID.metadata.xlsx', 'README.md.txt', '.DS_Store', 'Viral Pneumonia']


In [17]:
!ls -l /content/COVID-19_Radiography_Dataset/

ls: cannot access '/content/COVID-19_Radiography_Dataset/': No such file or directory


In [18]:


# Load each metadata file into Pandas DataFrames
covid_metadata = pd.read_excel(f'{data_dir}/COVID.metadata.xlsx')
lung_opacity_metadata = pd.read_excel(f'{data_dir}/Lung_Opacity.metadata.xlsx')
normal_metadata = pd.read_excel(f'{data_dir}/Normal.metadata.xlsx')
viral_pneumonia_metadata = pd.read_excel(f'{data_dir}/Viral Pneumonia.metadata.xlsx')

# # Inspect the first few rows of each DataFrame
# print("COVID Metadata:")
# display(covid_metadata.head(15))  # 'display' works well in Jupyter to show DataFrame outputs


In [19]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPU Name:", tf.test.gpu_device_name())

Num GPUs Available:  1
GPU Name: /device:GPU:0


In [ ]:
!nvidia-smi

Mon Oct  7 16:37:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              49W / 400W |    423MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [25]:
# Assuming your image directory is organized like /data_dir/class1/images/image1.jpg, etc.
classes = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']

# List all image file paths and labels
file_paths = []
labels = []

# Loop through each class and get all image paths
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name, 'images')  # Path to each class folder, include 'images' subfolder
    image_paths = glob.glob(os.path.join(class_dir, '*.png'))  # Change *.png to the actual image format, e.g., *.jpg
    file_paths.extend(image_paths)  # Add image paths to the list
    labels.extend([class_name] * len(image_paths))  # Add the corresponding labels

# Print the number of collected paths and labels
print(f"Number of file paths: {len(file_paths)}")
print(f"Number of labels: {len(labels)}")

# Create dataframe
df = pd.DataFrame({
    'file_path': file_paths,
    'label': labels
})

# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)
df.shape

# Now you have your dataframe with image paths and corresponding labels
print(df.head())

Number of file paths: 21165
Number of labels: 21165
                                           file_path         label
0  /content/COVID-19_Radiography_Dataset /Normal/...        Normal
1  /content/COVID-19_Radiography_Dataset /Normal/...        Normal
2  /content/COVID-19_Radiography_Dataset /Lung_Op...  Lung_Opacity
3  /content/COVID-19_Radiography_Dataset /Lung_Op...  Lung_Opacity
4  /content/COVID-19_Radiography_Dataset /Lung_Op...  Lung_Opacity


In [35]:
# Step 1: Split into train+val and test sets (80% train+val, 20% test)
train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Step 2: Split the train+val set into train and validation sets (80% of train+val for training, 20% for validation)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, stratify=train_val_df['label'], random_state=42)

# The split sizes are:
# Train: 0.8 * 0.8 = 64% of the original dataset
# Validation: 0.8 * 0.2 = 16% of the original dataset
# Test: 20% of the original dataset

# Print the sizes to verify
print(f"Training set size: {train_df.shape[0]}")
print(f"Validation set size: {val_df.shape[0]}")
print(f"Test set size: {test_df.shape[0]}")

Training set size: 12699
Validation set size: 4233
Test set size: 4233


In [37]:


# Assuming your image directory is organized like /data_dir/class1/images/image1.jpg

classes = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']

# List all image file paths and labels
file_paths = []
labels = []

# Loop through each class and get all image paths
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name, 'images')  # Path to each class folder
    image_paths = glob.glob(os.path.join(class_dir, '*.png'))  # Adjust format (*.png) as needed
    file_paths.extend(image_paths)
    labels.extend([class_name] * len(image_paths))

# Create dataframe
df = pd.DataFrame({
    'file_path': file_paths,
    'label': labels
})

# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

In [38]:



# Assuming df contains the columns 'file_path' and 'label'
# If not, you'll need to load the dataframe with image paths and labels

# Feature extraction from images
def extract_features(image_path, image_size=(128, 128)):
    # Read image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"Image not found or unable to read: {image_path}")
        return None

    # Resize the image (only if the image is successfully loaded)
    try:
        image = cv2.resize(image, image_size)
    except Exception as e:
        print(f"Error resizing image: {image_path}, Error: {e}")
        return None

    # Initialize feature list
    features = []

    # Average pixel value
    features.append(np.mean(image))

    # Standard deviation of pixel values
    features.append(np.std(image))

    # Histogram of oriented gradients (HOG)
    gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=1)
    mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)

    # Append histogram of angles (HOG-like feature)
    features.extend(np.histogram(angle, bins=9, range=(0, 360))[0])

    return features

# Extract features from images in the DataFrame
X = []
y = []
valid_image_count = 0

print("Extracting features from images...")
for path, label in tqdm(zip(df['file_path'], df['label']), desc='Extracting features', total=len(df)):
    features = extract_features(path)

    if features is not None:
        X.append(features)
        y.append(label)
        valid_image_count += 1

# Debugging information
print(f"Number of valid images processed: {valid_image_count}")
print(f"Shape of X: {len(X)}")
print(f"Shape of y: {len(y)}")

# Convert to numpy arrays for model input
X = np.array(X)
y = np.array(y)

# Check the first few paths and attempt to load one image for debugging
print(df['file_path'].head())
test_image = cv2.imread(df['file_path'].iloc[0], cv2.IMREAD_GRAYSCALE)
if test_image is None:
    print("Failed to load image.")
else:
    print("Image loaded successfully.")


# Extract features for all images
print("Extracting features from images...")
X = np.array([extract_features(path) for path in tqdm(df['file_path'], desc='Extracting features')])
y = np.array(df['label'])

Extracting features from images...


Extracting features: 100%|██████████| 21165/21165 [00:44<00:00, 470.45it/s]


Number of valid images processed: 21165
Shape of X: 21165
Shape of y: 21165
0    /content/COVID-19_Radiography_Dataset /Normal/...
1    /content/COVID-19_Radiography_Dataset /Lung_Op...
2    /content/COVID-19_Radiography_Dataset /Normal/...
3    /content/COVID-19_Radiography_Dataset /Viral P...
4    /content/COVID-19_Radiography_Dataset /Viral P...
Name: file_path, dtype: object
Image loaded successfully.
Extracting features from images...


Extracting features: 100%|██████████| 21165/21165 [00:44<00:00, 470.67it/s]


In [39]:
from collections import Counter

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Encode labels
le = LabelEncoder()
y_train_res = le.fit_transform(y_train_res)
y_val = le.transform(y_val)
y_test = le.transform(y_test)

# Print class distributions after SMOTE and splitting
def print_class_distribution(y, label):
    print(f"\nClass distribution in {label}:")
    print(Counter(y))

print_class_distribution(y_train_res, "training set after SMOTE")
print_class_distribution(y_val, "validation set")
print_class_distribution(y_test, "test set")


# Check the shape and data type of the train and validation data before optimization
print(f"X_train_res shape: {X_train_res.shape}, dtype: {X_train_res.dtype}")
print(f"y_train_res shape: {y_train_res.shape}, dtype: {y_train_res.dtype}")
print(f"X_val shape: {X_val.shape}, dtype: {X_val.dtype}")
print(f"y_val shape: {y_val.shape}, dtype: {y_val.dtype}")


Class distribution in training set after SMOTE:
Counter({1: 6523, 2: 6523, 3: 6523, 0: 6523})

Class distribution in validation set:
Counter({2: 1631, 1: 962, 0: 579, 3: 215})

Class distribution in test set:
Counter({2: 2038, 1: 1203, 0: 723, 3: 269})
X_train_res shape: (26092, 11), dtype: float64
y_train_res shape: (26092,), dtype: int64
X_val shape: (3387, 11), dtype: float64
y_val shape: (3387,), dtype: int64


In [45]:

!pip install optuna
import optuna
# Optuna optimization for XGBoost
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'objective': 'multi:softprob',  # Multiclass classification
        'num_class': 4  # Set to number of classes
    }

    model = xgb.XGBClassifier(**param)
    model.fit(
        X_train_res, y_train_res,
        eval_set=[(X_val, y_val)],
        # early_stopping_rounds=10,
        verbose=False
    )

    val_pred = model.predict(X_val)
    val_accuracy = accuracy_score(y_val, val_pred)
    return val_accuracy

# Optuna study for hyperparameter tuning
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Get the best parameters
best_params = study.best_params
best_score = study.best_value

print(f"\nBest parameters: {best_params}")
print(f"Best validation score: {best_score}")

# Train the final model with the best parameters
best_params['objective'] = 'multi:softprob'
best_params['num_class'] = 4  # Set to the number of classes
final_model = xgb.XGBClassifier(**best_params)
final_model.fit(
    np.vstack((X_train_res, X_val)),
    np.hstack((y_train_res, y_val)),
    verbose=False
)

# Final evaluation using the best XGBoost model
y_pred = final_model.predict(X_test)
y_pred_classes = le.inverse_transform(y_pred)

# Final evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Print final class distribution in predictions
print_class_distribution(y_pred_classes, "final predictions")

[I 2024-10-08 10:48:17,018] A new study created in memory with name: no-name-159c9158-6741-4f95-ae79-0e337f1ed78b
[I 2024-10-08 10:48:28,395] Trial 0 finished with value: 0.6486566282846177 and parameters: {'max_depth': 6, 'learning_rate': 0.0027024264204291706, 'n_estimators': 800, 'min_child_weight': 5, 'subsample': 0.6884195863418079, 'colsample_bytree': 0.5017855356799421, 'gamma': 0.00047701213996566717}. Best is trial 0 with value: 0.6486566282846177.
[I 2024-10-08 10:48:31,940] Trial 1 finished with value: 0.6843814585178624 and parameters: {'max_depth': 5, 'learning_rate': 0.5721981396001049, 'n_estimators': 668, 'min_child_weight': 3, 'subsample': 0.8257587218778262, 'colsample_bytree': 0.7264109701288941, 'gamma': 0.004893935784556175}. Best is trial 1 with value: 0.6843814585178624.
[I 2024-10-08 10:48:43,399] Trial 2 finished with value: 0.6991437850605255 and parameters: {'max_depth': 9, 'learning_rate': 0.009232804464596327, 'n_estimators': 784, 'min_child_weight': 8, 'su


Best parameters: {'max_depth': 10, 'learning_rate': 0.0810059622156233, 'n_estimators': 991, 'min_child_weight': 1, 'subsample': 0.7736040391146599, 'colsample_bytree': 0.6020431754723866, 'gamma': 9.179177850327193e-06}
Best validation score: 0.714201358134042

Test accuracy: 0.7234
                 precision    recall  f1-score   support

          COVID       0.67      0.67      0.67       723
   Lung_Opacity       0.67      0.61      0.63      1203
         Normal       0.79      0.81      0.80      2038
Viral Pneumonia       0.63      0.75      0.68       269

       accuracy                           0.72      4233
      macro avg       0.69      0.71      0.70      4233
   weighted avg       0.72      0.72      0.72      4233


Class distribution in final predictions:
Counter({'Normal': 2094, 'Lung_Opacity': 1094, 'COVID': 721, 'Viral Pneumonia': 324})
